<a href="https://colab.research.google.com/github/mounishsjsu/266_Project/blob/master/266_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

In [ ]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession,SQLContext
import warnings
warnings.filterwarnings('ignore')
from datetime import datetime
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import matplotlib.ticker as mtick
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from pyspark.sql.types import IntegerType
import seaborn as sns
import re
from textblob import TextBlob

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
conf = SparkConf().setMaster("local").setAppName("Weather analysis")
from sklearn.metrics import mean_squared_error
pd.set_option('display.float_format', lambda x: '%.5f' % x)
sc = SparkContext(conf = conf)
spark = SparkSession(sc)

sqlContext = SQLContext(sc)


In [10]:
df = spark.read.option("header",'true').csv("/content/city_temperature.csv",inferSchema=True)

In [11]:
print(df.head(3))

[Row(Region='Africa', Country='Algeria', State=None, City='Algiers', Month=1, Day=1, Year=1995, AvgTemperature=64.2), Row(Region='Africa', Country='Algeria', State=None, City='Algiers', Month=1, Day=2, Year=1995, AvgTemperature=49.4), Row(Region='Africa', Country='Algeria', State=None, City='Algiers', Month=1, Day=3, Year=1995, AvgTemperature=48.8)]


In [12]:
print(df.dtypes)

#Convert the data into a pandas dataframe
df=df.toPandas()

#Choosing only the rows with India from the dataframe
data_India  = df[ (df['Country'] == 'India') & (df['Year'] < 2020)]

[('Region', 'string'), ('Country', 'string'), ('State', 'string'), ('City', 'string'), ('Month', 'int'), ('Day', 'int'), ('Year', 'int'), ('AvgTemperature', 'double')]
